In [6]:
from fuzzywuzzy import fuzz
import Levenshtein
import jellyfish
import pandas as pd
import operator
from multiprocessing import Pool
from collections import Counter
import re, string
import numpy as np
pd.set_option('display.max_rows', 500)

# LinkSight Location Matching Algo 

Objective: Write a string matching algo that can process 2000 Philippine location in under 2 minutes and return the correct result 95% of the time. Using n-grams method to speed up performance. N-grams are contiguous sequences of n items from a given sample of text or speech. Breaking words and phrases into n-grams is a technique for narrowing the search space for doing fuzzy matching, which is computationally expensive.

## Import Philippine Standard Geographic Code reference file

In [7]:
psgc = pd.read_csv('psgc-locations.csv.gz',compression="gzip")
psgc.candidate_terms = psgc.loc_tuple.str.encode('utf-8').str.split(",").apply(tuple)
psgc.head()

,loc_tuple,code,bgy,municity,prov,candidate_terms
0,"ilocos norte,prov,012800000",12800000,NaN,NaN,ILOCOS NORTE,"(ilocos norte, prov, 012800000)"
1,"adams,ilocos norte,municity,012801000",12801000,NaN,ADAMS,ILOCOS NORTE,"(adams, ilocos norte, municity, 012801000)"
2,"adams pob,adams,ilocos norte,bgy,012801001",12801001,ADAMS POB.,ADAMS,ILOCOS NORTE,"(adams pob, adams, ilocos norte, bgy, 012801001)"
3,"adams,adams,ilocos norte,bgy,012801001",12801001,ADAMS POB.,ADAMS,ILOCOS NORTE,"(adams, adams, ilocos norte, bgy, 012801001)"
4,"bacarra,ilocos norte,municity,012802000",12802000,NaN,BACARRA,ILOCOS NORTE,"(bacarra, ilocos norte, municity, 012802000)"


## Create N-Gram Table

Turn the reference file into a dictionary of n-grams and their associated loc-phrases.

The purpose of this is to narrow down the number of candidate terms for fuzzy matching. It takes too long to do fuzzy matching on all 55k+ locations in the reference dataset. Instead, we'll break the lowest level item in each location phrase into 2-part n-grams. We'll then create a dictionary in which the keys are the unique n-grams and the values are all the location phrases that contain at least one instance of the said n-gram in their primary (first) term.

In [8]:
#Helper function that creates NGrams. does not include spaces

def makeNgram(string,n):
    string = re.sub("[^a-zA-Z0-9]+","",string.lower())
    ngrams = []
    #for n in range(2,n_max+1):
    for i in range(0,len(string)-(n-1)):
        ngram = unicode(string[i:i+n])
        ngrams.append(ngram)
    return list(set(ngrams)) #return only the unique n-grams. the same string can have repeats

In [9]:
def generate_ngram_table(loc_tuples,n):
    
    # create the dict
    ngram_table = {}
    
    #for each unique location phrase
    
    for loc in loc_tuples:
        
        #take each unique part in that tuple and extract the n-grams
        
        first_item = loc[0]
            
        first_item = re.sub("[^a-zA-Z0-9]+",u"",first_item.lower())
            
        #for each of these parts, extract the n-grams
            
        #for i in range(0,len(first_item)-(n-1)):
            
            #ngram = first_item[i:i+n].lower()
            
        ngrams_in_first_item = makeNgram(first_item, n)

            #if the n-gram is not yet in the table, add it as a new key for which value is empty list
            
        for ngram in ngrams_in_first_item:
            
            if ngram not in ngram_table.keys():
                
                ngram_table[ngram] = {loc}

            else:
                
                ngram_table[ngram].add(loc)
                
    return ngram_table

In [10]:
loc_phrase_ngram_table = generate_ngram_table(list(psgc.candidate_terms.dropna(how="all").unique()),2)

In [11]:
len(loc_phrase_ngram_table)

956

# Matching algorithm

There are four functions that work together to produce the matching algo:

**searchThruReference** - Takes a single group of search terms as a tuple, narrows down the candidate matches from the reference file based on those that share common 2-part n-grams with the first term, runs fuzzy matching on these to return best N results based on scoring formula. 
- **Input**: A single tuple of of up to three search terms sorted from lowest to highest administrative level, a n-gram dictionary with candidate matches as values, and N as the number of top N results you want to retrieve. In the search terms, the first item is the name of the lowest administrative item in the list. The last item is always the administrative class of the lowest-level term-- either bgy (barangay), municity (municipality or city), province (prov). Ex: ("Fort Bonifacio","Taguig City","bgy"). 
- **Output**: A dataframe with search term, its top N matches and the similarity scores, PSG codes of each.

**searchThruShortlist** - Helper to `searchThruReference`. Calculates the similarity score between each set of search terms and all its candidate matches. Uses multiprocessing to speed up the process. 
- **Input**: A single set of search terms as a tuple and a list of tuples representing its candidate matches. 
- **Output**: A dataframe containing similiarity scores between a single search term and each of its possible matches. Returns a single row is exact match is found.
    
**scoreMatches** - Helper to `searchThruShortlist`. Provides the formula for calculating similarity score between a single set of search terms and a single candidate match in the reference file.
- **Input**: A single set of search terms as a tuple and a single candidate as a tuple. 
- **Output**: (search terms, candidate terms, candidate PSG code, similarity score)

**getMatches** - Applies above steps to a list of search strings and returns a dataframe of search terms and each of their top N matches. For exact matches, only returns a single match.
- **Input**: A list of search tuples, ngram dictionary, number of results you want returned for each near match.
- **Output**: Dataframe of many search tuples and each of their top 5 matches, PSG codes, similarity scores.


In [12]:
def searchThruReference(searchTuple,ngram_table,nresults):
    
    possibleMatches = []
    
    # turn the search string into ngrams based on the length of ngrams in the reference table
    
    n = len(ngram_table.keys()[0])
    
    ss_ngrams = []
        
    # get the unique n-grams in the first item in the search tuple
    
    ss_ngrams = set(makeNgram(searchTuple[0],n))
    
    for ngram in list(ss_ngrams):        
        
        # look each n-gram up in the dictionary of n-grams and the candidates that contain them.
        # add the values as possible matches to be scored
        
        if ngram in ngram_table:
            
            possibleMatches += ngram_table[ngram] 

    
    if len(possibleMatches) == 0:
        
        print "Nothing found"
        
        pass
            
    else:
    #eliminate the candidates that have very few n-grams in common with the search terms
    
        threshold = len(ss_ngrams)/3    

        mostPossible = [k for k, v in Counter(possibleMatches).items() if v >= threshold]

        # calculate similarity scores of search terms with each candidate

        results = pd.DataFrame(searchThruShortlist(searchTuple,mostPossible)).rename(columns={0:'source',1:'match',2:'psgc',3:'score'})

        # get the top N results. for candidate matches that are aliases of the same place, keep the highest scoring one

        topresults = results.sort_values(by="score",ascending=False).reset_index(drop=True).drop_duplicates("psgc",keep="first")[:nresults]

        return topresults 
        

In [13]:
def searchThruShortlist(searchTuple,shortlist):
    
    #find exact matches first
    
    exact_match = [candidate for candidate in shortlist if searchTuple == candidate[:-1]]
    
    if exact_match <> []:
        
        psgc_code = exact_match[0][-1]
        
        #exact matches result in perfect score and a single row returned
        
        return pd.DataFrame([(searchTuple, exact_match[0], psgc_code,100)])
                

    else:

        #pair searchString with each possible match

        candidate_pairs = []
        for candidateTuple in shortlist:
            candidate_pairs.append((searchTuple,candidateTuple))

        #use multiprocessing to run fuzzy matching
        pool2 = Pool(2) 
        results = pool2.map(scoreMatches, candidate_pairs)
        pool2.close()
        pool2.join()

        return results

In [14]:
def scoreMatches(tuplePairs,firstItemRatioWgt=0.6,otherItemsRatioWgt=.4,admLevelMatchWgt=1.15):
    
    #split both the searchString and candidateString into their name and interlevel components. 

    searchTuple, candidateTuple = tuplePairs
    searchTerms, searchAdm = searchTuple[:-1], searchTuple[-1]
    candidateTuple = tuplePairs[1]
    candidateTerms, candidateAdm, candidateCode = candidateTuple[:-2], candidateTuple[-2], candidateTuple[-1]

    #if a searchString and the candidate have the same administrative level, this improve the resulting score by a multiplier
    admLevelMatchScore = (admLevelMatchWgt if searchAdm == candidateAdm else 1)

    #check on jw distance ratio between the very first items in search term and candidate term
    firstItemRatio = jellyfish.jaro_winkler(unicode(searchTerms[0]),unicode(candidateTerms[0])) * 100
                
    #Create a weighted score for the match with weights for each input.
    
    if len(searchTerms) == 1:
        score = (firstItemRatio*(firstItemRatioWgt+otherItemsRatioWgt)) * admLevelMatchScore/admLevelMatchWgt
    else:
        otherItemsRatio = fuzz.ratio(" ".join(searchTerms[0:])," ".join(candidateTerms[0:]))
        score = ((firstItemRatio*firstItemRatioWgt) + (otherItemsRatio*otherItemsRatioWgt)) * admLevelMatchScore/admLevelMatchWgt
                 
    results = (searchTuple,candidateTuple,candidateCode,score)
    
    return results

Test on a single string:

In [15]:
searchThruReference((u"polilio",u"quezon",u"municity"),loc_phrase_ngram_table,10) #shortlist first, then check for exact

,source,match,psgc,score
0,"(polilio, quezon, municity)","(polillo, quezon, municity, 045636000)",045636000,93.771429
1,"(polilio, quezon, municity)","(polilio, cabiao, nueva ecija, bgy, 034904023)",034904023,69.565217
2,"(polilio, quezon, municity)","(liliw, laguna, municity, 043410000)",043410000,69.028571
3,"(polilio, quezon, municity)","(polilio, cabanatuan, nueva ecija, bgy, 034903...",034903063,67.826087
4,"(polilio, quezon, municity)","(pilion, polillo, quezon, bgy, 045636013)",045636013,67.701863
5,"(polilio, quezon, municity)","(pola, oriental mindoro, municity, 175210000)",175210000,66.900000
6,"(polilio, quezon, municity)","(pagbilao, quezon, municity, 045630000)",045630000,66.828571
7,"(polilio, quezon, municity)","(polo, mauban, quezon, bgy, 045627024)",045627024,66.173913
8,"(polilio, quezon, municity)","(jomalig, quezon, municity, 045621000)",045621000,65.542857
9,"(polilio, quezon, municity)","(pili, sariaya, quezon, bgy, 045645032)",045645032,64.857143


In [16]:
searchThruReference((u"agm",u"delfin albano",u"bgy"),loc_phrase_ngram_table,10) #shortlist first, then check for exact

,source,match,psgc,score
0,"(agm, delfin albano, bgy)","(aga, delfin albano, isabela, bgy, 023118001)",023118001,77.066667
1,"(agm, delfin albano, bgy)","(dagman, araceli, palawan, bgy, 175303004)",175303004,74.800000
2,"(agm, delfin albano, bgy)","(naguma, faire, cagayan, bgy, 021526021)",021526021,71.600000
4,"(agm, delfin albano, bgy)","(agsimao, tineg, abra, bgy, 140125001)",140125001,71.371429
5,"(agm, delfin albano, bgy)","(bagumbayan, locsin, albay, bgy, 050503004)",050503004,70.000000
6,"(agm, delfin albano, bgy)","(agmailig, libacao, aklan, bgy, 060409001)",060409001,69.900000
7,"(agm, delfin albano, bgy)","(dagami, maasin, iloilo, bgy, 063029020)",063029020,69.600000
8,"(agm, delfin albano, bgy)","(bagumbayan, malinao, albay, bgy, 050510022)",050510022,69.600000
10,"(agm, delfin albano, bgy)","(naguma, calbayog, samar, bgy, 086003107)",086003107,68.800000
11,"(agm, delfin albano, bgy)","(jaguimit, dueas, iloilo, bgy, 063017024)",063017024,68.700000


In [17]:
searchThruReference((u"fe",u"culasi","antique",u"municity"),loc_phrase_ngram_table,10) #shortlist first, then check for exact

,source,match,psgc,score
0,"(fe, culasi, antique, municity)","(fe, culasi, antique, bgy, 060606019)",060606019,86.956522
1,"(fe, culasi, antique, municity)","(fe, jamindan, capiz, bgy, 061906009)",061906009,68.521739
2,"(fe, culasi, antique, municity)","(san fernando, masbate, municity, 054118000)",054118000,60.533333
3,"(fe, culasi, antique, municity)","(san fernando, la union, municity, 013314000)",013314000,60.133333
4,"(fe, culasi, antique, municity)","(san felipe, zambales, municity, 037110000)",037110000,59.600000
5,"(fe, culasi, antique, municity)","(ferrol, romblon, municity, 175916000)",175916000,59.466667
6,"(fe, culasi, antique, municity)","(san fernando, camarines sur, municity, 051732...",051732000,58.133333
7,"(fe, culasi, antique, municity)","(fely, maconacon, isabela, bgy, 023117003)",023117003,57.739130
8,"(fe, culasi, antique, municity)","(san fernando, san jose, antique, bgy, 060613028)",060613028,57.507246
9,"(fe, culasi, antique, municity)","(ferlda, alegria, surigao del norte, bgy, 1667...",166701008,56.231884


In [18]:
searchThruReference((u"polillo",u"quezon",u"municity"),loc_phrase_ngram_table,10)

,source,match,psgc,score
0,"(polillo, quezon, municity)","(polillo, quezon, municity, 045636000)",045636000,100


In [19]:
def getMatches(searchTupleList,ngram_table,nresults):
    all_matches = []
    for searchTuple in searchTupleList:
        searchTuple = tuple([item.lower() for item in searchTuple])
        searchTupleMatches = pd.DataFrame(searchThruReference(searchTuple,ngram_table,nresults))
        all_matches.append(searchTupleMatches)
#    return all_matches    
    return pd.concat(all_matches,ignore_index=True)

In [20]:
sample_list = [(u"Ilocos Sur",u"prov"),
               (u"Fort Bonifacio",u"Taguig",u"bgy"),
               (u"Baguio City",u"Benguet",u"city"),
               (u"Dagatan","Lipa", "Batangas", u"prov"),
               (u"Zamboanga City",u"city"),
               (u"Zamboanga Sibugay",u"prov"),
               ("Aga", "Delfin Albano",u"Isabela","bgy"),
               ("Ahin","Ifugao","bgy"),
               ("Santa Catalina","Lubao","Pampanga","bgy"),
               ("Dampalit", "Malabon","bgy"),
               ("San Pablo","Lagun","city"),
               ("Bgy 105","Caloocan","bgy"),
               ("brgy pasong tamo","quezon","bgy"),
               ("Tagaytay","Cavite","bgy")]

In [21]:
getMatches(sample_list,
           ngram_table=loc_phrase_ngram_table,
           nresults=10)

,source,match,psgc,score
0,"(ilocos sur, prov)","(ilocos sur, prov, 012900000)",012900000,100.000000
1,"(fort bonifacio, taguig, bgy)","(fort bonifacio, taguig, bgy, 137607020)",137607020,100.000000
2,"(baguio city, benguet, city)","(baguio, benguet, municity, 141102000)",141102000,76.996047
3,"(baguio city, benguet, city)","(bagu, bakun, benguet, bgy, 141103002)",141103002,69.881423
4,"(baguio city, benguet, city)","(baguio, tayabas, quezon, bgy, 045647013)",045647013,68.300395
5,"(baguio city, benguet, city)","(baguinloc, anao, tarlac, bgy, 036901001)",036901001,63.451910
6,"(baguio city, benguet, city)","(baguio, davao, davao del sur, bgy, 112402007)",112402007,62.735178
7,"(baguio city, benguet, city)","(bagua ii, cotabato, cotabato, bgy, 129804007)",129804007,62.627894
8,"(baguio city, benguet, city)","(bagua i, cotabato, cotabato, bgy, 129804006)",129804006,61.592321
9,"(baguio city, benguet, city)","(bagutot, bacnotan, la union, bgy, 013303005)",013303005,61.592321


# Notes

## Advantages of this method:
- Improved accuracy.
- Can handle cases of missing location components or misidentified interlevels

## Disadvantages

- Doesn't work as well when secondary terms are baked into the first term. Ex when "Fort Bonifacio Taguig" is listed as a barangay and mistakenly containst he municipality as well.

## To do list:
- Still gotta improve speed
- Streamline final steps of exporting file
- Explore using NLTK library which has ready functions for ngrams
